# Készítsünk regressziós modellt: adatok előkészítése és vizualizálása

## **Lineáris regresszió tökök esetében - 2. lecke**
#### Bevezetés

Most, hogy rendelkezésedre állnak azok az eszközök, amelyekkel elkezdheted a gépi tanulási modellek építését a Tidymodels és a Tidyverse segítségével, készen állsz arra, hogy kérdéseket tegyél fel az adataiddal kapcsolatban. Amikor adatokkal dolgozol és gépi tanulási megoldásokat alkalmazol, rendkívül fontos, hogy megtanuld, hogyan tegyél fel megfelelő kérdéseket, hogy teljes mértékben kiaknázhasd az adathalmazod lehetőségeit.

Ebben a leckében megtanulod:

- Hogyan készítsd elő az adataidat a modellépítéshez.

- Hogyan használd a `ggplot2`-t az adatok vizualizálásához.

Az a kérdés, amelyre választ szeretnél kapni, meghatározza, hogy milyen típusú gépi tanulási algoritmusokat fogsz alkalmazni. Az általad kapott válasz minősége pedig nagyban függ az adataid természetétől.

Nézzük meg ezt egy gyakorlati példán keresztül.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Illusztráció: @allison_horst</figcaption>


<!--![Illusztráció: \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Illusztráció: \@allison_horst-->


## 1. Tökadatok importálása és a Tidyverse előhívása

A következő csomagokra lesz szükségünk, hogy feldolgozzuk ezt a leckét:

-   `tidyverse`: A [tidyverse](https://www.tidyverse.org/) egy [R csomaggyűjtemény](https://www.tidyverse.org/packages), amelyet arra terveztek, hogy a data science gyorsabb, egyszerűbb és szórakoztatóbb legyen!

A következőképpen telepítheted őket:

`install.packages(c("tidyverse"))`

Az alábbi szkript ellenőrzi, hogy rendelkezel-e a modul elvégzéséhez szükséges csomagokkal, és ha valamelyik hiányzik, telepíti azokat.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Most indítsuk el néhány csomagot, és töltsük be a [adatokat](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv), amelyeket ehhez a leckéhez biztosítottak!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Egy gyors `glimpse()` azonnal megmutatja, hogy vannak hiányzó értékek, valamint szöveges (`chr`) és numerikus (`dbl`) adatok keveréke. A `Date` karakter típusú, és van egy furcsa oszlop, amelyet `Package`-nek hívnak, ahol az adatok között keverednek a `sacks`, `bins` és más értékek. Az adatok valójában egy kicsit zűrösek 😤.

Valójában nem túl gyakori, hogy egy teljesen használatra kész adatállományt kapjunk, amelyből azonnal gépi tanulási modellt lehet készíteni. De ne aggódj, ebben a leckében megtanulod, hogyan készíts elő egy nyers adatállományt a standard R könyvtárak segítségével 🧑‍🔧. Emellett különböző technikákat is elsajátítasz az adatok vizualizálására. 📈📊
<br>

> Egy kis emlékeztető: A csőoperátor (`%>%`) logikai sorrendben hajt végre műveleteket, azáltal hogy egy objektumot továbbít egy függvénybe vagy kifejezésbe. Gondolhatsz a csőoperátorra úgy, mintha a kódodban azt mondanád: "és aztán".


## 2. Hiányzó adatok ellenőrzése

Az egyik leggyakoribb probléma, amellyel az adatelemzők szembesülnek, az a hiányos vagy hiányzó adatok kezelése. Az R a hiányzó vagy ismeretlen értékeket egy speciális jelzőértékkel, `NA` (Not Available) jelöli.

De honnan tudhatjuk, hogy az adatkeret tartalmaz-e hiányzó értékeket?
<br>
-   Egy egyszerű módszer az alap R `anyNA` függvény használata, amely logikai objektumokat ad vissza: `TRUE` vagy `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Remek, úgy tűnik, hogy hiányoznak bizonyos adatok! Ez egy jó kiindulópont.

-   Egy másik módszer az `is.na()` függvény használata, amely logikai `TRUE` értékkel jelzi, hogy az adott oszlop elemei közül melyek hiányoznak.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Rendben, elvégeztem a feladatot, de egy ilyen nagy adatkeret esetében nem lenne hatékony és gyakorlatilag lehetetlen az összes sort és oszlopot egyenként átnézni😴.

-   Egy intuitívabb módszer az lenne, ha kiszámítanánk az egyes oszlopok hiányzó értékeinek összegét:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Sokkal jobb! Hiányoznak adatok, de lehet, hogy ez nem számít a feladat szempontjából. Nézzük meg, milyen eredményeket hoz a további elemzés.

> Az R nemcsak nagyszerű csomagokat és függvényeket kínál, hanem kiváló dokumentációval is rendelkezik. Például használja a `help(colSums)` vagy `?colSums` parancsot, hogy többet megtudjon a függvényről.


## 3. Dplyr: Az adatok manipulálásának nyelvtana

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Illusztráció: @allison_horst</figcaption>


<!--![Illusztráció: \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Illusztráció: \@allison_horst-->


A [`dplyr`](https://dplyr.tidyverse.org/), a Tidyverse egyik csomagja, egy adatmanipulációs nyelvtan, amely egy egységes igekészletet biztosít, hogy segítsen megoldani a leggyakoribb adatmanipulációs kihívásokat. Ebben a szakaszban megvizsgáljuk a dplyr néhány igéjét!  
<br>


#### dplyr::select()

A `select()` egy függvény a `dplyr` csomagban, amely segít kiválasztani, hogy mely oszlopokat tartsuk meg vagy zárjuk ki.

Ahhoz, hogy az adatkeretet könnyebben kezelhetővé tegyük, távolítsunk el néhány oszlopot a `select()` segítségével, és tartsuk meg csak azokat az oszlopokat, amelyekre szükségünk van.

Például ebben a gyakorlatban az elemzésünk a `Package`, `Low Price`, `High Price` és `Date` oszlopokat fogja érinteni. Válasszuk ki ezeket az oszlopokat.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

A `mutate()` egy függvény a `dplyr` csomagban, amely segít új oszlopokat létrehozni vagy meglévőket módosítani, miközben a meglévő oszlopokat megtartja.

A `mutate` általános szerkezete a következő:

`data %>%   mutate(new_column_name = what_it_contains)`

Nézzük meg, hogyan működik a `mutate` a `Date` oszlopon, az alábbi műveletek végrehajtásával:

1.  Az időpontokat (jelenleg karakter típusúak) hónap formátumra alakítjuk (ezek amerikai dátumok, tehát a formátum `MM/DD/YYYY`).

2.  Az időpontokból kinyerjük a hónapot egy új oszlopba.

Az R-ben a [lubridate](https://lubridate.tidyverse.org/) csomag megkönnyíti a dátum-idő adatokkal való munkát. Tehát használjuk a `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` függvényeket, hogy elérjük a fenti célokat. A `Date` oszlopot elhagyhatjuk, mivel a további műveletekhez már nem lesz rá szükség.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Hurrá! 🤩

Most hozzunk létre egy új `Price` oszlopot, amely a tök átlagos árát jelöli. Ezután vegyük a `Low Price` és a `High Price` oszlopok átlagát, hogy kitöltsük az új Price oszlopot.  
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Igen!💪

"De várjunk csak!", mondod majd, miután átfutottad az egész adathalmazt a `View(pumpkins)` segítségével, "Van itt valami furcsa!"🤔

Ha megnézed a `Package` oszlopot, a tökök sokféle kiszerelésben kerülnek értékesítésre. Néhányat `1 1/9 bushel` mértékegységben árulnak, néhányat `1/2 bushel` mértékegységben, néhányat darabra, néhányat súlyra (fontban), és vannak nagy dobozokban is, amelyek szélessége változó.

Ellenőrizzük ezt:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Csodás!👏

Úgy tűnik, hogy a tökök súlyának következetes mérése nagyon nehéz, ezért szűrjük őket úgy, hogy csak azokat a tököket válasszuk ki, amelyekben a *bushel* szó szerepel a `Package` oszlopban, és tegyük ezt egy új adatkeretbe, `new_pumpkins`.


#### dplyr::filter() és stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): létrehoz egy adat-alhalmazt, amely csak azokat a **sorokat** tartalmazza, amelyek megfelelnek a feltételeidnek, ebben az esetben azokat a tököket, amelyek *bushel* szöveget tartalmaznak a `Package` oszlopban.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): érzékeli egy minta jelenlétét vagy hiányát egy szövegben.

A [`stringr`](https://github.com/tidyverse/stringr) csomag egyszerű függvényeket biztosít gyakori szövegkezelési műveletekhez.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Láthatod, hogy körülbelül 415 sorra szűkítettük le az adatokat, amelyek tököt tartalmaznak zsákszámra.🤩  
<br>


#### dplyr::case_when()

**De várj! Van még egy dolog, amit meg kell tenned**

Észrevetted, hogy a véka mennyisége soronként változik? Normalizálnod kell az árképzést, hogy az árakat vékánként mutasd, ne pedig 1 1/9 vagy 1/2 vékánként. Ideje egy kis matematikát alkalmazni, hogy standardizáld.

A [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) függvényt fogjuk használni, hogy *módosítsuk* az Ár oszlopot bizonyos feltételek alapján. A `case_when` lehetővé teszi, hogy több `if_else()` állítást vektorizálj.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Most már elemezhetjük az egységárakat a bushel mértékegység alapján. Mindez a tökök busheljeinek tanulmányozása azonban azt mutatja, hogy mennyire `fontos` a `adataink természetének megértése`!

> ✅ A [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) szerint egy bushel súlya a termény típusától függ, mivel ez egy térfogatmérték. "Egy bushel paradicsom például 56 fontot kell, hogy nyomjon... A levelek és zöldek több helyet foglalnak kevesebb súllyal, így egy bushel spenót csak 20 font." Ez elég bonyolult! Ne foglalkozzunk a bushel-font átváltással, inkább árazzunk bushel alapján. Mindez a tökök busheljeinek tanulmányozása azonban azt mutatja, hogy mennyire fontos az adataink természetének megértése!
>
> ✅ Észrevetted, hogy a fél bushel tökök nagyon drágák? Ki tudod találni, miért? Tipp: a kis tökök sokkal drágábbak, mint a nagyok, valószínűleg azért, mert sokkal több van belőlük bushelként, tekintve az egy nagy üreges pite tök által elfoglalt kihasználatlan helyet.


Most végül, csak a kaland kedvéért 💁‍♀️, helyezzük át a Month oszlopot az első pozícióba, azaz a `Package` oszlop `elé`.

A `dplyr::relocate()` függvényt használjuk az oszlopok sorrendjének megváltoztatására.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Szép munka!👌 Most már van egy tiszta, rendezett adathalmazod, amelyre felépítheted az új regressziós modelledet!  
<br>


## 4. Adatvizualizáció ggplot2-vel

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografika: Dasani Madipalli</figcaption>


<!--![Infografika: Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Van egy *bölcs* mondás, ami így szól:

> "Az egyszerű grafikon több információt hozott a data analyst számára, mint bármely más eszköz." --- John Tukey

Az adatelemzők egyik feladata, hogy bemutassák az általuk vizsgált adatok minőségét és jellegét. Ehhez gyakran készítenek érdekes vizualizációkat, például diagramokat, grafikonokat és táblázatokat, amelyek az adatok különböző aspektusait mutatják be. Ily módon vizuálisan képesek megjeleníteni olyan összefüggéseket és hiányosságokat, amelyeket másképp nehéz lenne feltárni.

A vizualizációk abban is segíthetnek, hogy meghatározzuk, melyik gépi tanulási technika a legmegfelelőbb az adott adatokhoz. Például egy szórásdiagram, amely egy vonalat követ, azt jelezheti, hogy az adatok alkalmasak egy lineáris regressziós elemzésre.

Az R számos rendszert kínál grafikonok készítésére, de a [`ggplot2`](https://ggplot2.tidyverse.org/index.html) az egyik legelegánsabb és legsokoldalúbb. A `ggplot2` lehetővé teszi, hogy grafikonokat **független komponensek kombinálásával** állítsunk össze.

Kezdjünk egy egyszerű szórásdiagrammal, amely a Price és Month oszlopokat ábrázolja.

Ebben az esetben a [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html) függvénnyel kezdünk, megadunk egy adatkeretet és esztétikai leképezést (az [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html) segítségével), majd hozzáadunk rétegeket (például [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) a szórásdiagramokhoz.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Hasznos ez a grafikon 🤷? Meglepett rajta valami?

Nem különösebben hasznos, mivel csak annyit mutat, hogy az adataid pontokként szóródnak egy adott hónapban.
<br>


### **Hogyan tegyük hasznossá?**

Ahhoz, hogy a diagramok hasznos adatokat jelenítsenek meg, általában valahogyan csoportosítani kell az adatokat. Például a mi esetünkben, ha megtaláljuk a tökök havi átlagárát, az több betekintést nyújtana az adatok mögötti mintázatokba. Ez elvezet minket egy újabb **dplyr** bemutatóhoz:

#### `dplyr::group_by() %>% summarize()`

A csoportosított aggregáció R-ben könnyen kiszámítható a következővel:

`dplyr::group_by() %>% summarize()`

-   A `dplyr::group_by()` megváltoztatja az elemzés egységét a teljes adathalmazról az egyes csoportokra, például hónaponként.

-   A `dplyr::summarize()` egy új adatkeretet hoz létre, amelyben minden csoportosító változóhoz tartozik egy oszlop, valamint minden megadott összesítő statisztikához egy oszlop.

Például használhatjuk a `dplyr::group_by() %>% summarize()`-t arra, hogy a tököket a **Month** oszlop alapján csoportokba rendezzük, majd kiszámítsuk az egyes hónapok **átlagárát**.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Tömören!✨

A kategóriális jellemzők, mint például a hónapok, jobban ábrázolhatók oszlopdiagram segítségével 📊. Az oszlopdiagramokért felelős rétegek a `geom_bar()` és a `geom_col()`. További információért nézd meg a `?geom_bar` dokumentációját.

Készítsünk egyet!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 Ez egy sokkal hasznosabb adatvizualizáció! Úgy tűnik, hogy a legmagasabb tökárak szeptemberben és októberben fordulnak elő. Ez megfelel az elvárásaidnak? Miért igen, vagy miért nem?

Gratulálok a második leckéhez 👏! Előkészítetted az adataidat a modellépítéshez, majd további betekintéseket fedeztél fel vizualizációk segítségével!



---

**Felelősségkizárás**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével készült. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt a professzionális, emberi fordítás igénybevétele. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
